In [13]:
import sys
sys.path.append("/Users/aaron/phd/impact-of-contamination-on-taxonomy")

In [14]:
from workflow.fastani_contig_split.random_d_report_results import FastAniContigSplitRandomReportResultsFastAni
from collections import Counter

In [15]:

DF_RESULTS = FastAniContigSplitRandomReportResultsFastAni(replicate_id=1).output().read_cached()


# Subset the results to bacteria only
DF_RESULTS = DF_RESULTS[DF_RESULTS['expected_tax'].apply(lambda x: x.startswith('d__Bacteria'))]

DF_RESULTS.head()

[06/01/2023 14:26:04] - 15.3MiB = /srv/home/uqamussi/projects/gunc-chimeras/output/fastani_contig_split/random/r_1/results_report_k16_s5000__p50.tsv -> /tmp/gunc-cache/srv/home/uqamussi/projects/gunc-chimeras/output/fastani_contig_split/random/r_1/results_report_k16_s5000__p50.tsv


,gid,expected_tax,keep_tax,disc_tax,keep_rank_agree,disc_rank_agree,keep_sp_rep,disc_sp_rep,keep_ani,disc_ani,keep_af,disc_af,keep_type,disc_type,keep_same_as_207,disc_same_as_207,keep_same_as_disc
0,GCA_000143435.1,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,s,s,GCF_001435955.1,GCF_001435955.1,97.1201,97.4980,0.7887,0.7966,sp_rep,sp_rep,True,True,True
1,GCA_000153745.1,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.0000,no_ani,no_ani,True,True,True
2,GCA_000155005.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,s,NaN,GCF_003697165.2,0.0000,96.5249,0.0000,0.7589,no_af,sp_rep,False,True,False
3,GCA_000161815.1,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,s,s,GCF_001681715.1,GCF_001681715.1,97.1064,96.9964,0.9141,0.8293,sp_rep,sp_rep,True,True,True
4,GCA_000166735.2,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.0000,no_ani,no_ani,True,True,True


In [16]:
# Report how many of the keep halves are the same as R207
N_KEEP_SAME_R207 = len(DF_RESULTS[DF_RESULTS['keep_same_as_207'] == True])
N_KEEP_DIFF_R207 = len(DF_RESULTS[DF_RESULTS['keep_same_as_207'] == False])
N_KEEP_DIFF_R207_BUT_DISC_SAME =  len(DF_RESULTS[(DF_RESULTS['keep_same_as_207'] == False) & (DF_RESULTS['disc_same_as_207'] == True)])
print(f'There are {N_KEEP_SAME_R207:,} ({N_KEEP_SAME_R207/len(DF_RESULTS):.2%}) keep halves that are identical to the expected R207 taxonomy.')
print(f'There are {N_KEEP_DIFF_R207:,} ({N_KEEP_DIFF_R207/len(DF_RESULTS):.2%}) keep halves that are different to the expected R207 taxonomy.')
print(f'Of those, there are {N_KEEP_DIFF_R207_BUT_DISC_SAME:,} where the chimeric half has the expected R207 taxonomy (we picked the wrong half?)')
print('-' * 80)

N_DISC_SAME_R207 = len(DF_RESULTS[DF_RESULTS['disc_same_as_207'] == True])
N_DISC_DIFF_R207 = len(DF_RESULTS[DF_RESULTS['disc_same_as_207'] == False])
print(f'There are {N_DISC_SAME_R207:,} ({N_DISC_SAME_R207/len(DF_RESULTS):.2%}) chimeric halves that are identical to the expected R207 taxonomy.')
print(f'There are {N_DISC_DIFF_R207:,} ({N_DISC_DIFF_R207/len(DF_RESULTS):.2%}) chimeric halves that are different to the expected R207 taxonomy.')

print('-' * 80)

N_EITHER_DIFF = len(DF_RESULTS[(DF_RESULTS['keep_same_as_207'] == False) | (DF_RESULTS['disc_same_as_207'] == False)])
print(f'There are {N_EITHER_DIFF:,} ({N_EITHER_DIFF/len(DF_RESULTS):.2%}) cases were either half has an incorrect taxonomic assignment')

print('-' * 80)

N_ONE_RIGHT_ONE_WRONG = len(DF_RESULTS[
    ((DF_RESULTS['keep_same_as_207'] == True) & (DF_RESULTS['disc_same_as_207'] == False)) |
     ((DF_RESULTS['keep_same_as_207'] == False) & (DF_RESULTS['disc_same_as_207'] == True))
])
print(f'There are {N_ONE_RIGHT_ONE_WRONG:,} ({N_ONE_RIGHT_ONE_WRONG/len(DF_RESULTS):.2%}) cases where one half was wrong, and the other half was correct.')

N_NEITHER_CORRECT = len(DF_RESULTS[(DF_RESULTS['disc_same_as_207'] == False) & (DF_RESULTS['keep_same_as_207'] == False)])
print(f'There are {N_NEITHER_CORRECT:,} cases where neither half is correct.')

print('-' * 80)

N_KEEP_EQ_DISC = len(DF_RESULTS[DF_RESULTS['keep_same_as_disc'] == True])
print(f'There are {N_KEEP_EQ_DISC:,} ({N_KEEP_EQ_DISC/len(DF_RESULTS):.2%}) halves that are identical in assignment to each other.')


print('-' * 80)
N_BOTH_SAME = len(DF_RESULTS[(DF_RESULTS['keep_same_as_207'] == True) & (DF_RESULTS['disc_same_as_207'] == True)])
print(f'There are {N_BOTH_SAME:,} ({N_BOTH_SAME/len(DF_RESULTS):.2%})  hits where both halves have the same taxonomy')


There are 34,577 (99.23%) keep halves that are identical to the expected R207 taxonomy.
There are 269 (0.77%) keep halves that are different to the expected R207 taxonomy.
Of those, there are 187 where the chimeric half has the expected R207 taxonomy (we picked the wrong half?)
--------------------------------------------------------------------------------
There are 34,594 (99.28%) chimeric halves that are identical to the expected R207 taxonomy.
There are 252 (0.72%) chimeric halves that are different to the expected R207 taxonomy.
--------------------------------------------------------------------------------
There are 439 (1.26%) cases were either half has an incorrect taxonomic assignment
--------------------------------------------------------------------------------
There are 357 (1.02%) cases where one half was wrong, and the other half was correct.
There are 82 cases where neither half is correct.
-------------------------------------------------------------------------------

In [17]:
CNT_KEEP_HIGHEST_AGREE_TAX = Counter(DF_RESULTS[DF_RESULTS['keep_same_as_207'] == False]['keep_rank_agree'])
print('The following ranks are the highest rank at which the keep set differ from the expected tax:')
print(CNT_KEEP_HIGHEST_AGREE_TAX)

print('-' * 80)

CNT_DISC_HIGHEST_AGREE_TAX = Counter(DF_RESULTS[DF_RESULTS['disc_same_as_207'] == False]['disc_rank_agree'])
print('The following ranks are the highest rank at which the chimeric set differ from the expected tax:')
print(CNT_DISC_HIGHEST_AGREE_TAX)

# The discard set seems to be more skewed towards not finding a match.

The following ranks are the highest rank at which the keep set differ from the expected tax:
Counter({nan: 175, 'g': 93, 'f': 1})
--------------------------------------------------------------------------------
The following ranks are the highest rank at which the chimeric set differ from the expected tax:
Counter({nan: 176, 'g': 76})


In [18]:
# How many of the keep set had a chimeric half that yielded the correct taxonomy?
DF_KEEP_DIFF_BUT_DISC_SAME = DF_RESULTS[(DF_RESULTS['keep_same_as_207'] == False) & (DF_RESULTS['disc_same_as_207'] == True)]
N_KEEP_DIFF_BUT_DISC_SAME = len(DF_KEEP_DIFF_BUT_DISC_SAME)
print(f'There were {N_KEEP_DIFF_BUT_DISC_SAME:,} hits where the keep half was wrong, but the chimeric half was correct')

DF_DISAGREE_TAX = DF_KEEP_DIFF_BUT_DISC_SAME[DF_KEEP_DIFF_BUT_DISC_SAME['keep_tax'].apply(lambda x: len(x) > 7 if str(x) != 'nan' else False)]
print(f'Of those, there were {len(DF_DISAGREE_TAX):,} cases where the keep tax actually had an assignment')

DF_KEEP_DIFF_BUT_DISC_SAME

There were 187 hits where the keep half was wrong, but the chimeric half was correct
Of those, there were 79 cases where the keep tax actually had an assignment


,gid,expected_tax,keep_tax,disc_tax,keep_rank_agree,disc_rank_agree,keep_sp_rep,disc_sp_rep,keep_ani,disc_ani,keep_af,disc_af,keep_type,disc_type,keep_same_as_207,disc_same_as_207,keep_same_as_disc
2,GCA_000155005.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,s,NaN,GCF_003697165.2,0.0000,96.5249,0.0000,0.7589,no_af,sp_rep,False,True,False
190,GCA_000761495.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,g,NaN,GCF_014769185.1,NaN,96.2694,0.0000,0.8218,0.0000,sp_rep,no_ani,False,True,False
333,GCA_001063255.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,s,NaN,GCA_003028315.1,0.0000,96.7600,0.0000,0.9183,no_ani,sp_rep,False,True,False
622,GCA_001399545.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,s,NaN,GCF_016806145.1,0.0000,95.7276,0.0000,0.8461,no_ani,sp_rep,False,True,False
624,GCA_001399595.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,s,NaN,GCF_016806145.1,0.0000,95.4624,0.0000,0.8341,no_ani,sp_rep,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34534,GCF_900445985.1,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,NaN,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,NaN,s,NaN,GCF_017086445.1,0.0000,95.1800,0.0000,0.8746,no_ani,sp_rep,False,True,False
34955,GCF_900509435.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,s,NaN,GCF_900978845.1,0.0000,95.6917,0.0000,0.8191,no_ani,sp_rep,False,True,False
35241,GCF_900978905.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,s,NaN,GCF_900978845.1,0.0000,95.4564,0.0000,0.8650,no_ani,sp_rep,False,True,False
35572,GCF_902729445.1,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,NaN,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,NaN,s,NaN,GCF_001650635.1,0.0000,95.3563,0.0000,0.8567,no_ani,sp_rep,False,True,False


In [19]:
DF_NONE_CORRECT = DF_RESULTS[(DF_RESULTS['keep_same_as_207'] == False) & (DF_RESULTS['disc_same_as_207'] == False)]
print(f'There were {len(DF_NONE_CORRECT):,} cases where neither half got the correct taxonomy')

DF_NONE_CORRECT

There were 82 cases where neither half got the correct taxonomy


,gid,expected_tax,keep_tax,disc_tax,keep_rank_agree,disc_rank_agree,keep_sp_rep,disc_sp_rep,keep_ani,disc_ani,keep_af,disc_af,keep_type,disc_type,keep_same_as_207,disc_same_as_207,keep_same_as_disc
720,GCA_001603295.1,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.0000,no_ani,no_ani,False,False,True
1485,GCA_002348105.1,d__Bacteria;p__Deferribacterota;c__Deferribact...,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.0000,no_af,no_af,False,False,True
1801,GCA_002418505.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,g,g,GCA_002364755.1,GCF_008710035.1,95.1165,95.0947,0.8535,0.8251,sp_rep,sp_rep,False,False,False
1916,GCA_002450535.1,d__Bacteria;p__Marinisomatota;c__Marinisomatia...,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.0000,no_ani,no_ani,False,False,True
1925,GCA_002452365.1,d__Bacteria;p__Desulfobacterota;c__Syntrophia;...,NaN,d__Bacteria;p__Desulfobacterota;c__Syntrophia;...,NaN,g,NaN,GCA_000747625.2,0.0000,95.0709,0.0000,0.6682,no_ani,sp_rep,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26981,GCF_009493995.1,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,NaN,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,NaN,g,NaN,GCF_000157935.1,0.0000,95.8227,0.0000,0.7224,no_ani,sp_rep,False,False,False
29130,GCF_013417155.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,g,NaN,GCF_000487975.1,NaN,96.2424,0.0000,0.8764,0.0000,sp_rep,no_ani,False,False,False
30594,GCF_015667745.1,d__Bacteria;p__Actinobacteriota;c__Coriobacter...,NaN,d__Bacteria;p__Actinobacteriota;c__Coriobacter...,NaN,g,NaN,GCA_018382125.1,0.0000,95.3313,0.0000,0.8768,no_ani,sp_rep,False,False,False
31385,GCF_016806285.1,d__Bacteria;p__Campylobacterota;c__Campylobact...,d__Bacteria;p__Campylobacterota;c__Campylobact...,d__Bacteria;p__Campylobacterota;c__Campylobact...,g,g,GCF_900478295.1,GCF_003637665.1,95.5558,95.5284,0.9540,0.9142,sp_rep,sp_rep,False,False,False


In [20]:
DF_TO_INVESTIGATE = DF_RESULTS[(DF_RESULTS['keep_same_as_207'] == False) | (DF_RESULTS['disc_same_as_207'] == False)]
DF_TO_INVESTIGATE

,gid,expected_tax,keep_tax,disc_tax,keep_rank_agree,disc_rank_agree,keep_sp_rep,disc_sp_rep,keep_ani,disc_ani,keep_af,disc_af,keep_type,disc_type,keep_same_as_207,disc_same_as_207,keep_same_as_disc
2,GCA_000155005.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,s,NaN,GCF_003697165.2,0.0000,96.5249,0.0000,0.7589,no_af,sp_rep,False,True,False
150,GCA_000508605.1,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,NaN,s,NaN,GCF_000148585.2,NaN,95.4109,0.0000,0.8559,0.0000,sp_rep,no_ani,True,False,False
190,GCA_000761495.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,g,NaN,GCF_014769185.1,NaN,96.2694,0.0000,0.8218,0.0000,sp_rep,no_ani,False,True,False
333,GCA_001063255.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,s,NaN,GCA_003028315.1,0.0000,96.7600,0.0000,0.9183,no_ani,sp_rep,False,True,False
622,GCA_001399545.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,s,NaN,GCF_016806145.1,0.0000,95.7276,0.0000,0.8461,no_ani,sp_rep,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34955,GCF_900509435.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,s,NaN,GCF_900978845.1,0.0000,95.6917,0.0000,0.8191,no_ani,sp_rep,False,True,False
35241,GCF_900978905.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,s,NaN,GCF_900978845.1,0.0000,95.4564,0.0000,0.8650,no_ani,sp_rep,False,True,False
35572,GCF_902729445.1,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,NaN,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,NaN,s,NaN,GCF_001650635.1,0.0000,95.3563,0.0000,0.8567,no_ani,sp_rep,False,True,False
35612,GCF_902845755.1,d__Bacteria;p__Campylobacterota;c__Campylobact...,d__Bacteria;p__Campylobacterota;c__Campylobact...,d__Bacteria;p__Campylobacterota;c__Campylobact...,g,s,GCF_900478295.1,GCF_003637685.1,95.5244,95.4725,0.8923,0.9425,sp_rep,sp_rep,False,True,False


In [21]:
import pandas as pd

def get_tax_changes():
    rows = list()
    for row in DF_TO_INVESTIGATE.itertuples():

        expected_tax_split = row.expected_tax.split(';')

        cur_row = {
            'gid': row.gid,
            'expected_rank': row.expected_tax,
            'keep_tax': row.keep_type,
            'chim_tax': row.disc_type
        }

        if len(str(row.keep_tax)) > 10:
            keep_tax_split = row.keep_tax.split(';')

            # find the highest rank at which there is a disagreement
            for i, (e, k) in enumerate(zip(expected_tax_split, keep_tax_split)):
                if e != k:
                    cur_row['keep_tax'] = ';'.join(keep_tax_split[i:])
                    break
            else:
                cur_row['keep_tax'] = 'same'


        if len(str(row.disc_tax)) > 10:
            disc_tax_split = row.disc_tax.split(';')

            # find the highest rank at which there is a disagreement
            for i, (e, k) in enumerate(zip(expected_tax_split, disc_tax_split)):
                if e != k:
                    cur_row['chim_tax'] = ';'.join(disc_tax_split[i:])
                    break
            else:
                cur_row['chim_tax'] = 'same'

        rows.append(cur_row)

    return pd.DataFrame(rows)

DF_TAX_CHANGES = get_tax_changes()

DF_TAX_CHANGES.to_csv('/tmp/sp_clustering_tax_changes.tsv', sep='\t')

DF_TAX_CHANGES

,gid,expected_rank,keep_tax,chim_tax
0,GCA_000155005.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,no_af,same
1,GCA_000508605.1,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,same,no_ani
2,GCA_000761495.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,s__Acinetobacter lwoffii_D,no_ani
3,GCA_001063255.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,no_ani,same
4,GCA_001399545.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,no_ani,same
...,...,...,...,...
434,GCF_900509435.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,no_ani,same
435,GCF_900978905.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,no_ani,same
436,GCF_902729445.1,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,no_ani,same
437,GCF_902845755.1,d__Bacteria;p__Campylobacterota;c__Campylobact...,s__Helicobacter pylori,same
